In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, sum as spark_sum, col, count

# Create a Spark session
spark = SparkSession.builder.appName("Soccer Analysis").getOrCreate()

goalscorers_df = spark.read.csv("data/goalscorers.csv", header=True, inferSchema=True)
results_df = spark.read.csv("data/results.csv", header=True, inferSchema=True)

goalscorers_df.createOrReplaceTempView("goalscorers")
results_df.createOrReplaceTempView("results")

# 1) Number of goals scored by Lionel Messi (excluding own goals).

In [ ]:
# SQL
spark.sql('''
	SELECT COUNT(*) AS messi_goals
	FROM goalscorers 
	WHERE scorer = "Lionel Messi" AND own_goal = "FALSE"''').show()

+-----------+
|messi_goals|
+-----------+
|         55|
+-----------+



In [ ]:
# DataFrame API
goalscorers_df.filter(
   (col("scorer") == "Lionel Messi") & (col("own_goal") == False)
).count()

55

# 2) List of the 5 most recent matches played by the Spanish national team.

In [15]:
# SQL
spark.sql('''
	SELECT * FROM goalscorers 
	WHERE home_team = "Spain" OR away_team = "Spain" 
	ORDER BY date DESC 
	LIMIT 5''').show()

+----------+---------+---------+-------+---------------+------+--------+-------+
|      date|home_team|away_team|   team|         scorer|minute|own_goal|penalty|
+----------+---------+---------+-------+---------------+------+--------+-------+
|2024-07-14|    Spain|  England|  Spain|Mikel Oyarzabal|    86|   false|  false|
|2024-07-14|    Spain|  England|  Spain|  Nico Williams|    47|   false|  false|
|2024-07-14|    Spain|  England|England|    Cole Palmer|    73|   false|  false|
|2024-07-09|    Spain|   France|  Spain|      Dani Olmo|    25|   false|  false|
|2024-07-09|    Spain|   France|  Spain|   Lamine Yamal|    21|   false|  false|
+----------+---------+---------+-------+---------------+------+--------+-------+



In [17]:
# DataFrame API
goalscorers_df.filter(
   (col("home_team") == "Spain") | (col("away_team") == "Spain")
).orderBy(col("date").desc()).limit(5).show()

+----------+---------+---------+-------+---------------+------+--------+-------+
|      date|home_team|away_team|   team|         scorer|minute|own_goal|penalty|
+----------+---------+---------+-------+---------------+------+--------+-------+
|2024-07-14|    Spain|  England|  Spain|Mikel Oyarzabal|    86|   false|  false|
|2024-07-14|    Spain|  England|  Spain|  Nico Williams|    47|   false|  false|
|2024-07-14|    Spain|  England|England|    Cole Palmer|    73|   false|  false|
|2024-07-09|    Spain|   France|  Spain|      Dani Olmo|    25|   false|  false|
|2024-07-09|    Spain|   France|  Spain|   Lamine Yamal|    21|   false|  false|
+----------+---------+---------+-------+---------------+------+--------+-------+



# 3) Number of goals scored by Spain in its entire history. This information must be taken from results, as goalscorers does not contain all goals.

In [ ]:
# SQL
spark.sql('''
	SELECT 
		SUM(CASE WHEN home_team = 'Spain' THEN home_score ELSE 0 END) +
		SUM(CASE WHEN away_team = 'Spain' THEN away_score ELSE 0 END) AS total_spain_goals
	FROM results
''').show()

+-----------------+
|total_spain_goals|
+-----------------+
|             1567|
+-----------------+



In [ ]:
# DataFrame API
results_df.select(
	(sum(when(col("home_team") == "Spain", col("home_score")).otherwise(0)) +
	sum(when(col("away_team") == "Spain", col("away_score")).otherwise(0))
	).alias("total_spain_goals")
).show()

+-----------------+
|total_spain_goals|
+-----------------+
|             1567|
+-----------------+



# 4) List of the 5 highest scorers with the Spanish national team (excluding own goals).

In [ ]:
# SQL
spark.sql('''
	SELECT scorer, COUNT(*) AS goals
	FROM goalscorers
	WHERE team = 'Spain' AND own_goal = 'FALSE'
	GROUP BY scorer
	ORDER BY goals DESC
	LIMIT 5
''').show()

+---------------+-----+
|         scorer|goals|
+---------------+-----+
|    David Villa|   41|
|           Raúl|   32|
|  Álvaro Morata|   29|
|Fernando Torres|   28|
|Fernando Hierro|   25|
+---------------+-----+



In [26]:
# DataFrame API
goalscorers_df.filter(
    (col("team") == "Spain") & (col("own_goal") == "FALSE")
).groupBy("scorer") \
 .agg(count("*").alias("goals")) \
 .orderBy(col("goals").desc()) \
 .limit(5) \
 .show()

+---------------+-----+
|         scorer|goals|
+---------------+-----+
|    David Villa|   41|
|           Raúl|   32|
|  Álvaro Morata|   29|
|Fernando Torres|   28|
|Fernando Hierro|   25|
+---------------+-----+



# 5) List of Spanish players who have scored a penalty goal in a European Championship (UEFA Euro), sorted alphabetically.

In [28]:
# SQL
spark.sql('''
	SELECT DISTINCT scorer
	FROM results r
	JOIN goalscorers g
	ON r.date = g.date AND r.home_team = g.home_team AND r.away_team = g.away_team
	WHERE g.team = 'Spain' AND g.penalty = 'TRUE' AND r.tournament LIKE '%Euro%'
	ORDER BY scorer
''').show()

+--------------------+
|              scorer|
+--------------------+
|      Andrés Iniesta|
|         Daniel Ruiz|
|         David Villa|
|     Fernando Hierro|
|Francisco José Ca...|
|     Gaizka Mendieta|
|      José Claramunt|
|  Juan Antonio Señor|
|              Míchel|
|               Pirri|
|        Sergio Ramos|
|         Xabi Alonso|
|       Álvaro Morata|
+--------------------+



In [27]:
# DataFrame API

# Join results and goalscorers on date, home_team, and away_team
joined_df = results_df.join(
	goalscorers_df,
	on=["date", "home_team", "away_team"]
)
# Apply filters and select distinct scorers
joined_df.filter(
	(col("team") == "Spain") &
	(col("penalty") == "TRUE") &
	(col("tournament").like("%Euro%"))
).select("scorer").distinct().orderBy("scorer").show()

+--------------------+
|              scorer|
+--------------------+
|      Andrés Iniesta|
|         Daniel Ruiz|
|         David Villa|
|     Fernando Hierro|
|Francisco José Ca...|
|     Gaizka Mendieta|
|      José Claramunt|
|  Juan Antonio Señor|
|              Míchel|
|               Pirri|
|        Sergio Ramos|
|         Xabi Alonso|
|       Álvaro Morata|
+--------------------+



# 6) List of the 5 highest scorers in the final stages of the World Cup (FIFA World Cup) (excluding own goals).

In [29]:
# SQL
spark.sql('''
	SELECT scorer, COUNT(*) AS goals
	FROM results r
	JOIN goalscorers g
	ON r.date = g.date AND r.home_team = g.home_team AND r.away_team = g.away_team
	WHERE r.tournament = 'FIFA World Cup' AND g.own_goal = 'FALSE'
	GROUP BY scorer
	ORDER BY goals DESC
	LIMIT 5
''').show()

+--------------+-----+
|        scorer|goals|
+--------------+-----+
|Miroslav Klose|   16|
|       Ronaldo|   15|
|   Gerd Müller|   14|
| Just Fontaine|   13|
|  Lionel Messi|   13|
+--------------+-----+



In [30]:
# DataFrame API

# Using the joined DataFrame from the previous example
joined_df.filter(
	(col("tournament") == "FIFA World Cup") &
	(col("own_goal") == "FALSE")
).groupBy("scorer") \
 .agg(count("*").alias("goals")) \
 .orderBy(col("goals").desc()) \
 .limit(5) \
 .show()

+--------------+-----+
|        scorer|goals|
+--------------+-----+
|Miroslav Klose|   16|
|       Ronaldo|   15|
|   Gerd Müller|   14|
| Just Fontaine|   13|
|  Lionel Messi|   13|
+--------------+-----+



In [31]:
spark.stop()